# Generating a random sequence

We will build up to building a random sequence representing an unknown genome, but let's start simple. 

### A sequence of integers

Let's build a random sequence of integers first, using M integers to build a sequence of length L.

In [1]:
from random import randint

sequence = ''
length = 10
M=4

for i in range(length) :
    sequence += str(randint(1,M))
    
print (sequence)

2341334143


That was easy using the random integer generator function `randint()` that comes with Python. 

We used the `str()` function to convert the integers to strings and used the shorthand 'append' operator +=. 

### A sequence of letters

Let's step it up now and generate a random sequence of letters instead. We will do this by mapping integers to our aphabet. 


In [2]:
sequence = ''

alphabet = {1:'A', 2:'C', 3:'T', 4:'G'}

for i in range(length) :
    sequence += alphabet[randint(1,M)]
    
print (sequence)

TAATCATCTT


That went pretty smooth. All we needed was to define a **dictionary** that keeps the integer-to-letter mapping for our alphabet. 

Note that we did not need to import the randint function again because it is already in our namespace now. 

### A sequence of given composition

Unfortunately, this approach is limited. This is because randint returns **uniformly distributed** integers. In practice, we usually want to generate random sequences mimicing a given composition. 

Let's say we want a random sequence with  the following composition :
  * %15 A
  * %25 C
  * %50 T
  * %10 G
  

In [3]:
from random import random

sequence = ''

for i in range(length) :
    dice = random()
    if dice < 0.15 :
        sequence += 'A'
    elif dice < 0.4 :
        sequence += 'C'
    elif dice < 0.9 :
        sequence += 'T' 
    else :
        sequence += 'G'
        
print (sequence)

TTTGAAATCG


This works correctly but it is a long way from usable code. We write this kind of code where we **hard-code** variables as a **proof of concept**. It may be ok fore a quick experimentation but certainly not for practical usage. 

### Generalizing the proof of concept

We have an alphabet and composition, we can use a dictionary to hold them both nicely. We should then do the partitioning of the [0,1) interval algorithmically. 



In [4]:
composition = {'A':0.15, 'C':0.25, 'T':0.50, 'G':0.1}

sequence = ''

alphabet = composition.keys()

bound = 0 
bounds = []
for char in alphabet :
    bound += composition[char]
    bounds.append(bound)

print (bounds)

for i in range (length) :
    dice = random()
    for char, bound  in zip (alphabet, bounds) :
        if dice < bound :
            sequence+=char
            break
    
print (sequence)

[0.15, 0.4, 0.9, 1.0]
TTATTCGATT


Note that his works for any composition and alphabet, we only need to provide a dictionary representing the composition and integer representing a length.

### Make a function

Now that we have such a general version, we can make it a function so we can freely use it without any code duplication. Instead of printing, we should **return** the sequence so the user may do what s/he wants to do with it.

In [5]:
def MakeRandomSequence(composition, length) : 
    sequence = ''

    alphabet = composition.keys()

    bound = 0 
    bounds = []
    for char in alphabet :
        bound += composition[char]
        bounds.append(bound)

    assert 0.99< bounds[-1] < 1.01 
    
    for i in range (length) :
        dice = random()
        for char, bound  in zip (alphabet, bounds) :
            if dice < bound :
                sequence+=char
                break
    
    return sequence
    

Note the assert statement. We want to make sure the function refuses to run if the user provides faulty input. 

Now with the function it is very easy to change our **parameters**. 


In [6]:
composition = {'A':0.3, 'B':0.7}
print (MakeRandomSequence(composition, 20))

BBBBABAAABABBABBBABA


It will also give us a warning if the input is not sane :


In [7]:
composition = {'A':0.3, 'B':0.6}
print (MakeRandomSequence(composition, 20))

AssertionError: 

Don't be ciscouraged by the error, this is **good behavior**. If only all code was written this way. 

### Writing to a fasta file

Now, printing the output is ok for debugging, but we can't really process a random sequence with our eyes, especially a longer one. We also probably want to do something with the output later. Genomic sequence data is usually kept in **fasta** format. We can use the Biopyhon library to read/write this format.

Let's make use of our new function and write a large random genome to a fasta file

In [8]:
from Bio.Seq import Seq
from Bio import SeqIO

composition = {'A':0.15, 'C':0.25, 'T':0.50, 'G':0.1}

sequence = Seq(MakeRandomSequence(composition, 1000))

record = SeqIO.SeqRecord( sequence, 'random_sequence')

SeqIO.write(record, 'random.fasta', 'fasta')


1

Now, let's go look at the actual file we have written. 